# 02. LLM Synthetic Data Generation

이 노트북은 LLM을 사용하여 합성 쿼리 데이터를 생성합니다.

## 전제조건
먼저 `01_neural_sparse_base_training.ipynb`를 실행하여 기본 데이터를 생성해야 합니다.

## 목표
- LLM 모델 로딩 (Qwen3-30B-A3B-Thinking-2507-FP8)
- 한국어 문서 기반 합성 쿼리 생성
- 동의어 확장 및 검증

## 출력 데이터
모든 데이터는 `dataset/llm_generated/` 디렉토리에 저장됩니다:
- `synthetic_qd_pairs.pkl`: LLM 생성 Query-Document 쌍
- `enhanced_synonyms.json`: LLM 검증 동의어 사전

## 다음 단계
이 노트북 실행 후 `03_llm_enhanced_training.ipynb`를 실행하세요.

In [2]:
# Triton 컴파일 비활성화 (ARM aarch64 호환성)
import os

os.environ["TRITON_INTERPRET"] = "1"
os.environ["DISABLE_TRITON"] = "1"

print("✅ Triton compilation disabled for ARM compatibility")

✅ Triton compilation disabled for ARM compatibility


In [3]:
# DatasetManager 초기화
from src.dataset_manager import DatasetManager

dm = DatasetManager(base_path="dataset")
print("✅ DatasetManager initialized")
print(f"📁 Base path: {dm.base_path.absolute()}")

✅ DatasetManager initialized
📁 Base path: /home/west/Documents/cursor-workspace/opensearch-neural-pre-train/dataset


In [4]:
# 필수 데이터 파일 확인
required_files = [
    ("base_model", "documents.json"),
    ("base_model", "bilingual_synonyms.json"),
]

if not dm.check_dependencies(required_files):
    raise RuntimeError(
        "Required data files not found. "
        "Please run 01_neural_sparse_base_training.ipynb first."
    )

print("\n✅ Ready to proceed with LLM data generation")

✅ All dependencies satisfied

✅ Ready to proceed with LLM data generation


In [5]:
# Notebook 1에서 생성된 데이터 로드
print("Loading data from notebook 1...\n")

documents = dm.load_json("documents.json", "base_model")
bilingual_dict = dm.load_json("bilingual_synonyms.json", "base_model")

print(f"\n✅ Loaded {len(documents)} documents")
print(f"✅ Loaded bilingual dictionary with {len(bilingual_dict)} entries")

Loading data from notebook 1...

✓ Loaded JSON: dataset/base_model/documents.json
✓ Loaded JSON: dataset/base_model/bilingual_synonyms.json

✅ Loaded 127910 documents
✅ Loaded bilingual dictionary with 32 entries
✓ Loaded JSON: dataset/base_model/documents.json
✓ Loaded JSON: dataset/base_model/bilingual_synonyms.json

✅ Loaded 127910 documents
✅ Loaded bilingual dictionary with 32 entries


## 🆕 13. LLM 모델 로딩 및 초기화

이 섹션부터는 LLM 기반 확장 기능입니다:
- 합성 Query-Document 쌍 생성
- 한영 동의어 자동 검증
- 성능 개선

In [6]:
# Section 13: LLM Model Loading
import os

# Disable Triton JIT compilation to avoid Python.h dependency
os.environ["TRITON_DISABLE_LINE_INFO"] = "1"
os.environ["TRITON_CACHE_DIR"] = "/tmp/triton_cache"
os.environ["TRITON_INTERPRET"] = "1"  # 인터프리터 모드 사용
os.environ["DISABLE_TRITON"] = "1"    # 완전 비활성화

print("="*70)
print("🤖 섹션 13: LLM 모델 로딩 및 초기화")
print("="*70)

from src.llm_loader import load_qwen3_awq, check_gpu_memory

# GPU 메모리 체크
check_gpu_memory()

# Qwen3-30B-A3B-Thinking-2507-FP8 모델 로딩 (FP8 quantization, ARM compatible)
# Qwen3-30B-A3B-Thinking-2507-FP8 모델 로딩 (FP8 quantization, ARM compatible)
print("⚠️  참고: 첫 실행 시 모델 다운로드로 시간이 걸릴 수 있습니다.")
print("   모델 크기: ~30GB (FP8 quantization, ARM aarch64 compatible)")

llm_model, llm_tokenizer = load_qwen3_awq(
    model_name="Qwen/Qwen3-30B-A3B-Thinking-2507-FP8",
    device_map="auto",
)

print("\n✅ LLM 모델 로딩 완료!")
print("   모델을 사용하여 합성 데이터를 생성할 준비가 되었습니다.")

🤖 섹션 13: LLM 모델 로딩 및 초기화
🖥️  GPU Memory Status

GPU 0: NVIDIA GB10
  Total:       119.70 GB
  Allocated:     0.00 GB
  Free:        119.70 GB
⚠️  참고: 첫 실행 시 모델 다운로드로 시간이 걸릴 수 있습니다.
   모델 크기: ~30GB (FP8 quantization, ARM aarch64 compatible)

📥 Loading Qwen3 Model (FP8 Quantization)
Model: Qwen/Qwen3-30B-A3B-Thinking-2507-FP8
Device map: auto
💡 Using FP8 quantization (ARM aarch64 compatible)

1️⃣ Loading tokenizer...


/home/west/Documents/cursor-workspace/opensearch-neural-pre-train/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:435: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  queued_call()


✓ Tokenizer loaded (vocab size: 151,669)

2️⃣ Loading model with FP8 quantization...
   Using Transformers native FP8 support (no external deps needed)
   This may take a few minutes...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✓ Model loaded

📊 GPU Memory after loading: 30.22 GB

✅ Qwen3 Model Ready!

✅ LLM 모델 로딩 완료!
   모델을 사용하여 합성 데이터를 생성할 준비가 되었습니다.


## 🆕 14. LLM 기반 합성 Query-Document Pairs 생성

LLM을 사용하여 문서로부터 검색 쿼리를 역생성합니다.
이를 통해 학습 데이터를 대폭 확장할 수 있습니다.

In [ ]:
print("\n" + "="*70)
print("📝 섹션 14: LLM 기반 합성 Query-Document Pairs 생성")
print("="*70)

from src.synthetic_data_generator import generate_synthetic_qd_pairs

# 기존 문서에서 합성 쿼리 생성
# 처음 1000개 문서 사용 (시간 고려)
print(f"\n📊 합성 데이터 생성 설정:")
print(f"   - 사용할 문서: 1,000개")
print(f"   - 문서당 쿼리: 3개")
print(f"   - 예상 생성 pairs: ~3,000개")
print(f"   - 예상 소요 시간: ~15-30분 (GPU 속도에 따라)")

synthetic_pairs = generate_synthetic_qd_pairs(
    documents=documents[:1000],  # 처음 1000개 문서
    llm_model=llm_model,
    llm_tokenizer=llm_tokenizer,
    num_queries_per_doc=3,
    batch_size=2,
    max_documents=1000,
    enable_filtering=True,
)

print(f"\n✅ 합성 데이터 생성 완료: {len(synthetic_pairs):,}개 pairs")

# 샘플 출력
print("\n📋 생성된 합성 데이터 샘플 (처음 5개):")
print("="*70)
for i, (query, doc, relevance) in enumerate(synthetic_pairs[:5], 1):
    print(f"\n{i}. Query: {query}")
    print(f"   Document: {doc[:100]}...")
    print(f"   Relevance: {relevance}")
print("="*70)


📝 섹션 14: LLM 기반 합성 Query-Document Pairs 생성

📊 합성 데이터 생성 설정:
   - 사용할 문서: 1,000개
   - 문서당 쿼리: 3개
   - 예상 생성 pairs: ~3,000개
   - 예상 소요 시간: ~15-30분 (GPU 속도에 따라)

📝 Generating Synthetic Query-Document Pairs
Start time: 2025-11-15 18:42:31
Documents: 1000
Queries per doc: 3
Expected total queries: 3000
Quality filtering: ON
Verbose logging: ON


Generating queries:   0%|                                                  | 0/1000 [00:00<?, ?it/s]


📊 Progress Report - Document 1/1000
⏱️  Elapsed: 0.0m | ETA: 0.0m
✅ Generated: 0 pairs
❌ Failed/Filtered: 0
⚡ Avg time/doc: 0.00s

📄 Doc 1: Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters...
   🤖 Calling LLM...


Generating queries:   0%|                                                  | 0/1000 [05:28<?, ?it/s]


KeyboardInterrupt: 

: 

## 🆕 15. LLM 기반 한영 동의어 검증 및 확장

기존 임베딩 기반으로 발견한 동의어를 LLM으로 검증하여 정확도를 높입니다.

In [ ]:
print("\n" + "="*70)
print("🌐 섹션 15: LLM 기반 한영 동의어 검증 및 확장")
print("="*70)

from src.cross_lingual_synonyms import enhance_bilingual_dict_with_llm

# 기존 임베딩 기반 동의어를 LLM으로 검증
print(f"\n📊 동의어 검증 설정:")
print(f"   - 기존 동의어 사전: {len(bilingual_dict):,}개 항목")
print(f"   - 검증할 항목: 상위 100개")
print(f"   - 검증 임계값: 0.8")
print(f"   - 예상 소요 시간: ~5-10분")

enhanced_bilingual_dict = enhance_bilingual_dict_with_llm(
    initial_dict=bilingual_dict,  # Cell 14에서 생성된 기존 사전
    llm_model=llm_model,
    llm_tokenizer=llm_tokenizer,
    verification_threshold=0.8,
    max_verify=100,  # 상위 100개만 검증 (시간 절약)
)

print(f"\n✅ LLM 검증 완료!")
print(f"   기존 사전: {len(bilingual_dict):,}개")
print(f"   검증 후: {len(enhanced_bilingual_dict):,}개")

# 검증 결과 샘플 출력
print("\n📋 검증된 동의어 샘플:")
print("="*70)
sample_count = 0
for korean, english_list in list(enhanced_bilingual_dict.items())[:10]:
    sample_count += 1
    print(f"{sample_count}. {korean} ↔ {', '.join(english_list)}")
print("="*70)

# 업데이트된 동의어 사전을 bilingual_dict에 반영
bilingual_dict = enhanced_bilingual_dict
print(f"\n✓ 동의어 사전 업데이트 완료")

---

# 데이터 저장 (Notebook 간 공유)

LLM으로 생성된 데이터를 `dataset/llm_generated/` 디렉토리에 저장합니다.
다음 노트북에서 이 데이터를 로드하여 enhanced 모델을 학습할 수 있습니다.

In [ ]:
# 1. LLM 생성 Query-Document 쌍 저장
dm.save_pickle(
    synthetic_pairs,
    "synthetic_qd_pairs.pkl",
    "llm_generated"
)

In [ ]:
# 2. LLM 검증 동의어 사전 저장
dm.save_json(
    enhanced_bilingual_dict,
    "enhanced_synonyms.json",
    "llm_generated"
)

In [ ]:
# 저장된 데이터 요약
dm.print_summary()

---

## ✅ Notebook 2 완료

모든 LLM 생성 데이터가 `dataset/llm_generated/`에 저장되었습니다.

### 다음 단계
이제 `03_llm_enhanced_training.ipynb`를 실행하여 enhanced 모델을 학습하고 성능을 비교하세요.